# 11번가 크롤링

In [123]:
import requests
import numpy as np
import pandas as pd

In [124]:
# 11번가에서 키워드에 대한 판매량 순 정렬 결과를 크롤링

def _11st_df(kwd, top_num):           # kwd: key word,   top_num: 상위 몇개의 결과를 가져올지
    base_url='https://search.11st.co.kr/Search.tmall'
    params={'kwd': kwd,
            'pageNo': 1,
            'pageSize': 100,
            'sortCd': 'A',  # sort condition
            'method': 'getSearchFilterAjax'
           }
    response=requests.get(base_url,params=params)
    doc=response.json()  # 해당 문서는 json
    page=10
    result_list=[]

    def remove_comma(x):  # 가격 data에서 ,를 제외하고 문자열을 join한 후 int type으로 만드는 함수 생성
        result="".join(x.split(','))
        return int(result)
    
    for i in range(1,page+1):
        for i in range(len(doc['commonPrdList']['items'])):
            title=doc['commonPrdList']['items'][i]['prdNm']
            price=doc['commonPrdList']['items'][i]['finalPrc']
            
        # 무료배송인 경우일 때 '무료배송'표시가 아니라 배송비용에 대한 표기가 없는 data를 처리하기 위해 try-except를 활용
            try:
                if doc['commonPrdList']['items'][i]['deliveryPriceText']=='무료배송':
                     shipping_charge='무'
                else:
                    shipping_charge='유'
            except:
                    shipping_charge='무'
            
        # 별점 data가 없는 경우 nan입력
            try:
                star_point=doc['commonPrdList']['items'][i]['buySatisfyScore']
            except:
                star_point=np.nan
            
            review_count=doc['commonPrdList']['items'][i]['reviewCount']
    
            result_list.append({
                'title': title,
                'price': remove_comma(price),
                'shipping_charge': shipping_charge,
                'star_point': star_point,
                'review_count': review_count
            })
        
        result_df=pd.DataFrame(result_list)
        
        # 크롤링한 결과의 품목에서 관심 대상(해당 기업의 전자제품)이 아닌 호환 악세서리, 관련 없는 제품 등의 품목을 제외
        stop_word = ['USB', '충전기', '케이스', '케이블', '페이퍼', 'A4', '잉크', '꽁짜폰', '기저귀',
            '사과', '파우치', '앱스토어', '기프트카드', '필름', '요금제', '주스', '거름망',
            '세탁망', '토너', 'CLT', '스트랩', '드라이버', '거치대', '플레이 스토어', '잠옷']
        
        # stop_word에 해당하는 단어가 제품명에 포함되어 있을경우 그 index를 반환한다.
        # stop_word에 해당하는 단어가 제품명에 포함되어 있지 않을 경우 -1을 반환한다.
        # 따라서 반환값이 -1이 아닐 경우에 해당 data를 drop.
        for i in range(len(result_df['title'])):
            for j in stop_word:
                if result_df.loc[i, 'title'].find(j) != -1:
                    result_df.drop(index=i, inplace=True)
                    break
                    
        return result_df.head(top_num)

## 고찰

1. Json type의 크롤링에서 헤매었다.
> Json type의 경우 Network 탭에서 response를 확인해 request를 추적해야 한다.

2. 크롤링한 data에서 일관되지 않은 data에 대한 처리 - try-except 활용
>(무료배송이지만 '무료배송'표기가 해당 css selector에 저장된 것이 아니라 해당 css selector 자체가 없는 경우가 몇개 존재했다)
>
>(별점 data가 존재하지 않는 품목이 존재했다)

3. '112,450'과 같은 가격표기 방식을 integer type으로 변환하기 위해 remove_comma() 함수 생성
> ','를 기준으로 split()한 후 join -> int type 변환
>
> data frame의 형태이므로 pd.to_numeric을 활용 하는 방법도 있었다.

4. stop_word의 부족함
> 충분히 불필요한 data가 filtering 됐다고 보기 어렵다.(상위 결과를 보았을 때는 양호해 보였으나)
>
> NLP에 대한 추가학습 필요성

sample